In [1]:
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error, mean_absolute_error
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout
from tensorflow.keras.callbacks import EarlyStopping
import ta
import yfinance as yf
from datetime import datetime

2025-05-12 21:32:20.448843: I tensorflow/core/util/port.cc:110] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-05-12 21:32:20.473963: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F AVX512_VNNI AVX512_BF16 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2025-05-12 21:32:21.199311: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [2]:
print(tf.__version__); 
print(tf.config.list_physical_devices('GPU'))

2.13.0
[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]


2025-05-12 21:32:30.518723: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:981] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2025-05-12 21:32:30.539406: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:981] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2025-05-12 21:32:30.539581: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:981] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.


In [7]:
physical_devices = tf.config.list_physical_devices('GPU')
if len(physical_devices) > 0:
    print("GPU 사용 중!")
else:
    print("GPU가 사용되지 않고 있습니다.")

gpus = tf.config.list_physical_devices('GPU')
for gpu in gpus:
    print(gpu)

GPU 사용 중!
PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')


In [4]:
# 완료된, 실패한 종목 관리 파일
completed_file = 'completed_symbols.txt'
failed_file = 'failed_symbols.txt'

# 기존 파일 읽기
completed_symbols = set()
if os.path.exists(completed_file):
    with open(completed_file, 'r') as f:
        completed_symbols = set(f.read().splitlines())

failed_symbols = set()
if os.path.exists(failed_file):
    with open(failed_file, 'r') as f:
        failed_symbols = set(f.read().splitlines())

In [8]:
# sMAPE 계산 함수
def smape(a, f):
    return 100 / len(a) * np.sum(2 * np.abs(f - a) / (np.abs(a) + np.abs(f)))

# 모델 저장 경로
MODEL_FOLDER = 'LSTM_MODEL'
os.makedirs(MODEL_FOLDER, exist_ok=True)

# 종목별 평가 지표 저장용
results = []

# 전체 CSV 로딩
df_all = pd.read_csv('sp500_latest.csv')
symbols = df_all['Symbol'].unique()


for symbol in symbols:

    try:
        df = df_all[df_all['Symbol'] == symbol].copy()
        df = df.dropna()

        # 기술적 지표 추가
        df['MA20'] = ta.trend.sma_indicator(df['Close'], window=20)
        bb = ta.volatility.BollingerBands(df['Close'], window=20, window_dev=2)
        df['Upper'] = bb.bollinger_hband()
        df['Lower'] = bb.bollinger_lband()
        df['RSI'] = ta.momentum.RSIIndicator(df['Close'], window=14).rsi()
        df.dropna(inplace=True)

        features = ['Open', 'High', 'Low', 'Close', 'Volume', 'MA20', 'Upper', 'Lower', 'RSI']
        scaler = MinMaxScaler()
        scaled_data = scaler.fit_transform(df[features])

        sequence_length = 50
        X, y = [], []
        for i in range(len(scaled_data) - sequence_length):
            X.append(scaled_data[i:i+sequence_length])
            y.append(scaled_data[i+sequence_length][features.index('Close')])
        X = np.array(X)
        y = np.array(y)

        # 7:2:1 split
        train_size = int(len(X) * 0.7)
        val_size = int(len(X) * 0.2)
        X_train, X_val, X_test = X[:train_size], X[train_size:train_size+val_size], X[train_size+val_size:]
        y_train, y_val, y_test = y[:train_size], y[train_size:train_size+val_size], y[train_size+val_size:]

        # 모델 구성
        model = Sequential([
            LSTM(64, return_sequences=True, input_shape=(X.shape[1], X.shape[2])),
            Dropout(0.3),
            LSTM(32),
            Dropout(0.3),
            Dense(1)
        ])
        model.compile(optimizer='adam', loss='mse')
        early_stop = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)

        # 학습
        model.fit(X_train, y_train, epochs=50, batch_size=32,
                  validation_data=(X_val, y_val), callbacks=[early_stop], verbose=0)

        # 예측 및 역변환
        pred = model.predict(X_test)
        close_index = features.index('Close')
        y_full = np.zeros((len(y_test), len(features)))
        pred_full = np.zeros((len(pred), len(features)))
        y_full[:, close_index] = y_test
        pred_full[:, close_index] = pred.flatten()
        true_rescaled = scaler.inverse_transform(y_full)[:, close_index]
        pred_rescaled = scaler.inverse_transform(pred_full)[:, close_index]

        # 평가
        smape_val = smape(true_rescaled, pred_rescaled)
        mae_val = mean_absolute_error(true_rescaled, pred_rescaled)
        mse_val = mean_squared_error(true_rescaled, pred_rescaled)

        results.append({
            'Symbol': symbol,
            'sMAPE': smape_val,
            'MAE': mae_val,
            'MSE': mse_val
        })

        # 모델 저장
        model.save(os.path.join(MODEL_FOLDER, f'{symbol}.h5'))
        # 성공시 기록
        with open(completed_file, 'a') as f:
            f.write(f"{symbol}\n")

        print(f"{symbol} 완료 - sMAPE: {smape_val:.2f}%, MAE: {mae_val:.2f}, MSE: {mse_val:.2f}")
        

    except Exception as e:
        print(f"{symbol} 실패: {e}")
        # 실패 기록
        with open(failed_file, 'a') as f:
            f.write(f"{symbol}\n")

# 결과 정리
results_df = pd.DataFrame(results)
results_df = results_df.sort_values('sMAPE')

# sMAPE 구간화
bins = np.arange(0, 105, 5)
labels = [f'{i}~{i+5}%' for i in bins[:-1]]
results_df['sMAPE_Group'] = pd.cut(results_df['sMAPE'], bins=bins, labels=labels, right=False)

print("\n--- Top 10 종목 (sMAPE 낮은 순) ---")
print(results_df.head(10))

print("\n--- sMAPE 오차 범위별 종목 개수 (5% 단위) ---")
print(results_df['sMAPE_Group'].value_counts().sort_index())

print("\n--- 전체 지표 ---")
print(results_df)

print("\nGPU 상태:")
print("TensorFlow version:", tf.__version__)
print("Num GPUs Available:", len(tf.config.list_physical_devices('GPU')))
print("Available devices:", tf.config.list_physical_devices())


8/8 [==============================] - 0s 3ms/step
RSG 완료 - sMAPE: 4.65%, MAE: 8.88, MSE: 94.16


/home/pyeon/cudaTest/tf_gpu_env/lib/python3.8/site-packages/keras/src/engine/training.py:3000: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


8/8 [==============================] - 1s 3ms/step
RMD 완료 - sMAPE: 3.10%, MAE: 6.42, MSE: 75.00


/home/pyeon/cudaTest/tf_gpu_env/lib/python3.8/site-packages/keras/src/engine/training.py:3000: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


8/8 [==============================] - 0s 4ms/step
RVTY 완료 - sMAPE: 2.14%, MAE: 2.40, MSE: 10.82


...

/home/pyeon/cudaTest/tf_gpu_env/lib/python3.8/site-packages/keras/src/engine/training.py:3000: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


8/8 [==============================] - 0s 5ms/step
ZBH 완료 - sMAPE: 1.14%, MAE: 1.30, MSE: 3.12


/home/pyeon/cudaTest/tf_gpu_env/lib/python3.8/site-packages/keras/src/engine/training.py:3000: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


8/8 [==============================] - 0s 5ms/step
ZTS 완료 - sMAPE: 1.97%, MAE: 3.42, MSE: 19.63

--- Top 10 종목 (sMAPE 낮은 순) ---
    Symbol     sMAPE       MAE       MSE sMAPE_Group
185    FRT  1.056653  1.087832  1.832881        0~5%
190     FE  1.066591  0.410212  0.263235        0~5%
495    ZBH  1.142575  1.300099  3.122340        0~5%
173   EVRG  1.191103  0.641156  0.688281        0~5%
158     EA  1.275970  1.834090  5.880165        0~5%
225   HOLX  1.359353  1.052106  1.811874        0~5%
464     VZ  1.361064  0.529494  0.488892        0~5%
168    EQR  1.361458  0.893344  1.347707        0~5%
387      O  1.370936  0.732864  0.869473        0~5%
2      ABT  1.371821  1.505809  3.834759        0~5%

--- sMAPE 오차 범위별 종목 개수 (5% 단위) ---
sMAPE_Group
0~5%       390
5~10%       85
10~15%      14
15~20%       4
20~25%       3
25~30%       1
30~35%       0
35~40%       0
40~45%       0
45~50%       0
50~55%       0
55~60%       0
60~65%       0
65~70%       0
70~75%       0
75~80%       0
8

/home/pyeon/cudaTest/tf_gpu_env/lib/python3.8/site-packages/keras/src/engine/training.py:3000: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


In [ ]:
# sMAPE, MAE, MSE 전체 평균
avg_sMAPE = results_df['sMAPE'].mean()
avg_MAE = results_df['MAE'].mean()
avg_MSE = results_df['MSE'].mean()

print(f'전체 평균 sMAPE: {avg_sMAPE:.2f}%')
print(f'전체 평균 MAE: {avg_MAE:.4f}')
print(f'전체 평균 MSE: {avg_MSE:.4f}')